<a href="https://colab.research.google.com/github/Naqvi-786/Dataset-Demo/blob/main/pre%20ignition%20work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [141]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [142]:
data = pd.read_excel('/content/cleaned_data.xlsx')

In [143]:
print(data.columns)

Index(['Experiment', 'time (s)', 'Within 60 s of Ignition', 'Pre-Ignition',
       'Pan TC center', 'Pan TC2', 'Pan TC3', 'Pan TC', 'Duct TC1',
       'Duct TC1 K', 'Hood TC', 'Velocity', 'ΔV Ch 1 Smoke', 'ΔV Ch 2 Alcohol',
       'ΔV Ch 3 HC's', 'ΔV Ch 4 HC's', 'ΔV Ch 5 H2', 'ΔV Ch 8 Nat Gas',
       'ΔV Ch 10 CO', 'ΔV Ch 12 VOCs', 'ΔV Dust Sensor 1', 'ΔV Dust Sensor 2',
       'ΔV Dust Sensor 3', 'ΔV Dust Sensor Final', '14 Humidity %vol',
       '18 Humidity %vol', '15CO2 PPM', '17CO2 PPM', 'ΔPPM 16 iAQ',
       'ΔPPM 18 CO'],
      dtype='object')


In [144]:
X = data.drop(columns=['Pre-Ignition'])  # Features
y = data['Pre-Ignition']  # Target

# Identify columns with non-numeric data
non_numeric_cols = X.select_dtypes(exclude=np.number).columns

# Drop or convert non-numeric columns
# Option 1: Drop non-numeric columns
X_numeric = X.drop(columns=non_numeric_cols)

# Option 2: Convert non-numeric to numeric using encoding (e.g., one-hot encoding)
# from sklearn.preprocessing import OneHotEncoder
# ... (encoding logic)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric) # Use the modified DataFrame

In [145]:
_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))


In [146]:
#X = X.iloc[:len(y)]

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [148]:
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

In [149]:
model = Sequential()

In [150]:
model.add(LSTM(64, activation='relu', input_shape=(X_train.shape[1], 1), return_sequences=True)) # Reshape input_shape
model.add(Dropout(0.2))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [151]:
model.add(Dense(1, activation='sigmoid'))


In [152]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [153]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [154]:
from sklearn.preprocessing import LabelEncoder

In [156]:
if y_train.dtype == 'object':
    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.transform(y_test)  # Transform y_test using the same encoder

In [157]:
if y_train.dtype == 'object':
    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.transform(y_test)

In [158]:
y_train = pd.to_numeric(y_train, errors='coerce')
y_test = pd.to_numeric(y_test, errors='coerce')

In [159]:
y_train = pd.to_numeric(y_train)
y_test = pd.to_numeric(y_test)

In [160]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_data=(X_test, y_test),
                    class_weight=class_weights,
                    callbacks=[early_stopping])

Epoch 1/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 15s 35ms/step - accuracy: 0.7837 - loss: 0.5181 - val_accuracy: 0.8417 - val_loss: nan
Epoch 2/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 20s 33ms/step - accuracy: 0.8769 - loss: 0.3125 - val_accuracy: 0.9102 - val_loss: nan
Epoch 3/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.9141 - loss: 0.2164 - val_accuracy: 0.9292 - val_loss: nan
Epoch 4/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - accuracy: 0.9258 - loss: 0.1861 - val_accuracy: 0.9436 - val_loss: nan
Epoch 5/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.9356 - loss: 0.1667 - val_accuracy: 0.9300 - val_loss: nan
Epoch 6/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.9447 - loss: 0.1431 - val_accuracy: 0.9319 - val_loss: nan
Epoch 7/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - accuracy: 0.9523 - loss: 0.1226 - val_accuracy: 0.9467 - val_loss: nan
Epoch 8/50
322/322 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.9554 - loss: 0.1257 - val_accuracy: 0.9627 

In [161]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8572 - loss: nan
Test Loss: nan
Test Accuracy: 0.8416958451271057


In [162]:
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [163]:
print(predictions[:10])

[[1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]]


In [164]:
!pip install scikeras

In [165]:
from scikeras.wrappers import KerasClassifier  # Import KerasClassifier from scikeras
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight

In [180]:
def create_model(lstm_units=64, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(lstm_units, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(lstm_units//2, activation='relu', return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [181]:
model = KerasClassifier(model=create_model, verbose=0)

In [182]:
!pip install scikeras --upgrade
!pip install --upgrade scikit-learn

In [183]:
model = KerasClassifier(
    model=create_model,  # Use 'model' instead of 'build_fn'
    epochs=50,
    batch_size=32,
    verbose=0
)

In [184]:
param_grid = {
    'lstm_units': [32, 64, 128],
    'dropout_rate': [0.2, 0.3, 0.5],
    'learning_rate': [0.0001, 0.001, 0.01],
    'batch_size': [32, 64],
    'epochs': [50, 100]
}

In [185]:
model._estimator_type = "classifier"

In [186]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

AttributeError: 'super' object has no attribute '__sklearn_tags__'